## Goal:

<insert Kelli's message from slack here>

## Imports

In [19]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import numpy as np

## Data read-in

In [52]:
df = pd.read_csv('CHI_agents_data-Table 1.csv',low_memory=False)

In [54]:
# Function to convert WKT format to a Point geometry
def to_point(coord):
    try:
        if isinstance(coord, str) and 'POINT' in coord:
            # Use WKT to convert the string into a Point object
            return wkt.loads(coord)
        else:
            return None  # Return None for invalid or missing coordinates
    except Exception as e:
        return None  # Catch any issues

# Apply the conversion to the 'geometry' column
df['geometry'] = df['geometry'].apply(to_point)

# Now convert the DataFrame to a GeoDataFrame
df_geo = gpd.GeoDataFrame(df, geometry='geometry')

In [55]:
boundaries_df = gpd.read_file('Boundaries - Community Areas (current).geojson')

In [56]:
communities_to_keep = [
    'NEAR SOUTH SIDE',
    'ARMOUR SQUARE',
    'BRIDGEPORT',
    'MCKINLEY PARK',
    'BRIGHTON PARK',
    'ARCHER HEIGHTS',
    'GARFIELD RIDGE',
    'CLEARING',
    'WEST ELSDON',
    'GAGE PARK',
    'NEW CITY',
    'FULLER PARK',
    'GRAND BOULEVARD',
    'DOUGLAS',
    'OAKLAND',
    'KENWOOD',
    'HYDE PARK',
    'WASHINGTON PARK',
    'ENGLEWOOD',
    'WEST ENGLEWOOD',
    'CHICAGO LAWN',
    'WEST LAWN',
    'WOODLAWN',
    'GREATER GRAND CROSSING',
    'SOUTH SHORE',
    'AUBURN GRESHAM',
    'ASHBURN',
    'CHATHAM',
    'AVALON PARK',
    'SOUTH CHICAGO',
    'CALUMET HEIGHTS',
    'BURNSIDE',
    'WASHINGTON HEIGHTS',
    'BEVERLY',
    'MOUNT GREENWOOD',
    'MORGAN PARK',
    'WEST PULLMAN',
    'SOUTH DEERING',
    'RIVERDALE',
    'EAST SIDE',
    'HEGEWISCH'
]

In [57]:
filtered_boundaries = boundaries_df[boundaries_df['community'].isin(communities_to_keep)]

In [58]:
df_geo = df_geo.set_crs(4326)

In [74]:
# Read in neighborhood boundaries
neighborhood_boundaries = filtered_boundaries

# Clean and convert the 'Price' column to integers
df_geo['Price_int'] = df_geo['Price'].str.replace('[\$,]', '', regex=True).astype(int)

# Ensure both GeoDataFrames (df_geo and neighborhood_boundaries) have the same CRS
if df_geo.crs != neighborhood_boundaries.crs:
    df_geo = df_geo.to_crs(neighborhood_boundaries.crs)  # Transform CRS to match

# Perform a spatial join to find which points (brokerages) fall within which community
joined_gdf = gpd.sjoin(df_geo, neighborhood_boundaries, how="inner", predicate='within')

# # Group by community and agent/team to calculate counts and total sales
# grouped = joined_gdf.groupby(['community', 'Final_Agent/Team']).agg(
#     count=('Final_Agent/Team', 'count'),
#     total_price=('Price_int', 'sum')
# ).reset_index().nlargest(20)

# # Find the agent/team with the highest count for each community
# max_count_idx = grouped.groupby("community")['count'].idxmax()
# agent_result = grouped.loc[max_count_idx, ['community', 'Final_Agent/Team', 'count', 'total_price']]
# agent_result = agent_result.rename(columns={'count': 'agent_count', 'total_price': 'agent_price'})

# # Calculate community-level totals
# community_result = grouped.groupby('community').agg(
#     community_count=('count', 'sum'),
#     community_price=('total_price', 'sum')
# ).reset_index()

# # Merge agent results with community-level totals
# merged_df = pd.merge(agent_result, community_result, on='community', how='inner')

# # Drop unnecessary columns from neighborhood boundaries
# columns_to_drop = ['area', 'shape_area', 'perimeter', 'area_num_1', 'area_numbe', 'comarea_id', 'comarea', 'shape_len']
# neighborhood_boundaries = neighborhood_boundaries.drop(columns=columns_to_drop, errors='ignore')

# # Merge with neighborhood boundaries
# final_merge = neighborhood_boundaries.merge(merged_df, on='community')
# final_merge = final_merge.rename(columns={'Final_Agent/Team': 'final_agent_team'})

# # Save the result to a GeoJSON file
# final_merge.to_file("final_merge.geojson", driver='GeoJSON')


In [85]:
joined_gdf

,mls_id,Address,Location,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,Brokerage,...,index_right,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len
6,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",buy_agent_1,buyer,Anne Fan,Anne Fan,Compass,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
7,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",list_agent_1,listing,George Toscas,George Toscas,ACO Commercial,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
8,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",list_agent_2,listing,Linda Hattar,Linda Hattar,ACO Commercial,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
126,11993747,4825 S Woodlawn Ave,CHI - Kenwood,60615.0,"$4,000,000",list_agent_1,listing,Eugene Fu,Eugene Fu,@properties Christie's International Real Estate,...,4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062
135,11993747,4825 S Woodlawn Ave,CHI - Kenwood,60615.0,"$4,000,000",buy_agent_1,buyer,Susan O'Connor,Susan O'Connor,Berkshire Hathaway HomeServices Chicago,...,4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108203,11836116,12910 S Parnell Ave,CHI - West Pullman,60628.0,"$100,000",list_agent_1,listing,Vera Brown,Vera Brown,R G Ramsey & Associates,...,51,WEST PULLMAN,0,99365198.0822,0,53,53,0,0,50023.8430008
108204,12041412,8222 S Perry Ave,CHI - Chatham,60620.0,"$100,000",list_agent_1,listing,Vernon Lilly,Vernon Lilly,EXIT True Design Realty LLC,...,39,CHATHAM,0,82320670.3112,0,44,44,0,0,42006.9450094
108208,11887086,7558 S Marshfield Ave,CHI - Auburn Gresham,60620.0,"$100,000",list_agent_1,listing,Victor Vita,Victor Vita,Vylla Home,...,69,AUBURN GRESHAM,0,105065353.602,0,71,71,0,0,46757.7217161
108210,11876519,6950 S Vernon Ave,CHI - Greater Grand Crossing,60637.0,"$100,000",list_agent_1,listing,William Bates Jr,William Bates Jr,EXIT Strategy Realty,...,66,GREATER GRAND CROSSING,0,98853167.7093,0,69,69,0,0,54645.3302996


In [101]:
top_agents_list = joined_gdf.value_counts('Final_Agent/Team').head(31)
top_agents_list = top_agents_list.keys().to_list()

In [104]:
joined_gdf.columns

Index(['mls_id', 'Address', 'Location', 'Zip_Code', 'Price', 'agent_no',
       'side', 'agent_name', 'Final_Agent/Team', 'Brokerage', 'url',
       'TRD_Note', 'geometry', 'SqFt', 'price_per_sq_ft', 'year_built',
       'broker_email', 'data_source_name', 'within_county', 'Price_int',
       'index_right', 'community', 'area', 'shape_area', 'perimeter',
       'area_num_1', 'area_numbe', 'comarea_id', 'comarea', 'shape_len'],
      dtype='object')

In [109]:
gdf_for_export = joined_gdf[['mls_id', 'Address', 'Location', 'Zip_Code', 'Price', 'agent_no',
       'side', 'agent_name', 'Final_Agent/Team', 'Brokerage', 'url',
       'TRD_Note', 'geometry', 'SqFt', 'price_per_sq_ft', 'year_built',
       'broker_email']]

In [111]:
# Create an ExcelWriter instance
with pd.ExcelWriter('all_agents_data.xlsx') as writer:
    for agent in top_agents_list:
        # Filter for the current agent
        x = gdf_for_export[gdf_for_export['Final_Agent/Team'] == agent]
        
        # Write each agent's dataframe to a different sheet
        x.to_excel(writer, sheet_name=agent, index=False)


In [112]:
joined_gdf

,mls_id,Address,Location,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,Brokerage,...,index_right,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len
6,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",buy_agent_1,buyer,Anne Fan,Anne Fan,Compass,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
7,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",list_agent_1,listing,George Toscas,George Toscas,ACO Commercial,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
8,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",list_agent_2,listing,Linda Hattar,Linda Hattar,ACO Commercial,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
126,11993747,4825 S Woodlawn Ave,CHI - Kenwood,60615.0,"$4,000,000",list_agent_1,listing,Eugene Fu,Eugene Fu,@properties Christie's International Real Estate,...,4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062
135,11993747,4825 S Woodlawn Ave,CHI - Kenwood,60615.0,"$4,000,000",buy_agent_1,buyer,Susan O'Connor,Susan O'Connor,Berkshire Hathaway HomeServices Chicago,...,4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108203,11836116,12910 S Parnell Ave,CHI - West Pullman,60628.0,"$100,000",list_agent_1,listing,Vera Brown,Vera Brown,R G Ramsey & Associates,...,51,WEST PULLMAN,0,99365198.0822,0,53,53,0,0,50023.8430008
108204,12041412,8222 S Perry Ave,CHI - Chatham,60620.0,"$100,000",list_agent_1,listing,Vernon Lilly,Vernon Lilly,EXIT True Design Realty LLC,...,39,CHATHAM,0,82320670.3112,0,44,44,0,0,42006.9450094
108208,11887086,7558 S Marshfield Ave,CHI - Auburn Gresham,60620.0,"$100,000",list_agent_1,listing,Victor Vita,Victor Vita,Vylla Home,...,69,AUBURN GRESHAM,0,105065353.602,0,71,71,0,0,46757.7217161
108210,11876519,6950 S Vernon Ave,CHI - Greater Grand Crossing,60637.0,"$100,000",list_agent_1,listing,William Bates Jr,William Bates Jr,EXIT Strategy Realty,...,66,GREATER GRAND CROSSING,0,98853167.7093,0,69,69,0,0,54645.3302996


In [130]:
joined_gdf['price_as_int']

6         $8000000
7         $8000000
8         $8000000
126       $4000000
135       $4000000
            ...   
108203     $100000
108204     $100000
108208     $100000
108210     $100000
108211     $100000
Name: price_as_int, Length: 13571, dtype: object

In [132]:
joined_gdf['price_as_int'] = joined_gdf['Price'].str.replace("$","",regex=False)
joined_gdf['price_as_int'] = joined_gdf['price_as_int'].str.replace(",","",regex=False).astype(int)

In [135]:
agents_by_transaction = joined_gdf.groupby('Final_Agent/Team')['price_as_int'].sum()

In [140]:
joined_gdf.columns

Index(['mls_id', 'Address', 'Location', 'Zip_Code', 'Price', 'agent_no',
       'side', 'agent_name', 'Final_Agent/Team', 'Brokerage', 'url',
       'TRD_Note', 'geometry', 'SqFt', 'price_per_sq_ft', 'year_built',
       'broker_email', 'data_source_name', 'within_county', 'Price_int',
       'index_right', 'community', 'area', 'shape_area', 'perimeter',
       'area_num_1', 'area_numbe', 'comarea_id', 'comarea', 'shape_len',
       'price_as_int'],
      dtype='object')

In [142]:
joined_gdf.groupby('Final_Agent/Team')['price_as_int'].sum()

Final_Agent/Team
Aaqila Harvey       569900
Aaron Campbell      127000
Aaron Franklin      252000
Aaron Gaines       4900147
Aaron Greenberg     455000
                    ...   
Zoe Wohlsifer       135000
Zoey Zhu           2459588
Zofia Strzep       1659800
Zofia Zon-Leiva     620100
Zorica Ledic        268000
Name: price_as_int, Length: 5279, dtype: int64

In [145]:
joined_gdf.groupby('Final_Agent/Team')['price_as_int'].sum()

Final_Agent/Team
Aaqila Harvey       569900
Aaron Campbell      127000
Aaron Franklin      252000
Aaron Gaines       4900147
Aaron Greenberg     455000
                    ...   
Zoe Wohlsifer       135000
Zoey Zhu           2459588
Zofia Strzep       1659800
Zofia Zon-Leiva     620100
Zorica Ledic        268000
Name: price_as_int, Length: 5279, dtype: int64

In [146]:
agents_by_transaction.sort_values(ascending=False).to_csv("agents_by_dollar_volume.csv")

In [153]:
top_199_agents_by_dollar_volume = agents_by_transaction.sort_values(ascending=False).head(199).keys().to_list()

In [154]:
# Create an ExcelWriter instance
with pd.ExcelWriter('all_agents_dollar_data.xlsx') as writer:
    for agent in top_199_agents_by_dollar_volume:
        # Filter for the current agent
        x = gdf_for_export[gdf_for_export['Final_Agent/Team'] == agent]
        
        # Write each agent's dataframe to a different sheet
        x.to_excel(writer, sheet_name=agent, index=False)


## Collect property types

In [258]:
to_be_sliced = joined_gdf.reset_index(drop=True)

In [263]:
group_1 = to_be_sliced.iloc[0:8000]
group_2 = to_be_sliced.iloc[8000:]

In [199]:
import requests

url = "https://redfin-com-data.p.rapidapi.com/properties/details"

# querystring = {"url":"/IL/Chicago/6605-S-Kimbark-Ave-60637/home/12570870"}

querystring = {"url":"/IL/Chicago/100-E-14th-St-60605/unit-1112/home/39569782"}

headers = {
    "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
    "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

In [200]:
response.json()['data']['belowTheFold']['publicRecordsInfo']['basicInfo']['propertyTypeName']

'Condo/Co-op'

In [202]:
joined_gdf['api_url'] = joined_gdf['url'].replace?

Object `replace` not found.


In [206]:
joined_gdf['api_url'] = joined_gdf['url'].str[22:]

In [209]:
# def get_property_type(api_url=str):
    
#     querystring = {"url":api_url}

#     headers = {
#         "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
#         "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
#     }

#     response = requests.get(url, headers=headers, params=querystring)
    
#     property_type = response.json()['data']['belowTheFold']['publicRecordsInfo']['basicInfo']['propertyTypeName']
    
#     return property_type

In [235]:
import time
import requests

def get_property_type_with_error(api_url, max_retries=3, backoff_factor=1.0):
    """
    Fetch property type from the Redfin API, with error handling and exponential backoff.

    Returns:
      - The property type (string), OR
      - An error message (string) if something goes wrong.
    """
    
    querystring = {"url": api_url}
    headers = {
        "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",  
        "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
    }

    attempt = 1
    while attempt <= max_retries:
        try:
            response = requests.get(url, headers=headers, params=querystring, timeout=10)
            response.raise_for_status()

            data = response.json()
            property_type = data['data']['belowTheFold']['publicRecordsInfo']['basicInfo']['propertyTypeName']
            
            return property_type
        
        except requests.exceptions.RequestException as re:
            # Networking or HTTP error
            if attempt == max_retries:
                # Return error message as a string
                return f"ERROR: Request failed after {max_retries} retries. {re}"
            else:
                # Exponential backoff
                delay = backoff_factor * (2 ** (attempt - 1))
                time.sleep(delay)
                attempt += 1

        except KeyError as ke:
            # JSON doesn't have the fields we expect
            return f"ERROR: Unexpected JSON structure. Missing key: {ke}"

        except Exception as e:
            # Catch-all
            return f"ERROR: An unexpected error occurred: {e}"

In [236]:
test_df = joined_gdf.tail(30)

In [241]:
from tqdm.auto import tqdm  # or from tqdm import tqdm
tqdm.pandas()              # This adds "progress_apply" to pandas DataFrames

In [271]:
test_df['property_type'] = test_df['api_url'].apply(get_property_type_with_error)

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:


# Instead of:
# test_df['property_type'] = test_df['api_url'].apply(get_property_type_with_error)
# do this:
# test_df['property_type'] = test_df['api_url'].progress_apply(get_property_type_with_error)
group_1['property_type'] = group_1['api_url'].progress_apply(get_property_type_with_error)
group_1.to_csv("group_1.csv")


  0%|          | 0/8000 [00:00<?, ?it/s]

In [ ]:
group_2['property_type'] = group_2['api_url'].progress_apply(get_property_type_with_error)
group_2.to_csv("group_2.csv")

In [270]:
group_1['property_type'].iloc[0]

"ERROR: An unexpected error occurred: 'NoneType' object is not subscriptable"